In [ ]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import helper
import models

import matplotlib as mpl
import pickle
from copy import deepcopy
import time
#from graphviz import Digraph
import itertools
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
import seaborn as sns
from scipy.stats import multivariate_normal

In [ ]:
SIGMA_R = .001
SIGMA_R_MODEL = .2

Tx = 5
Ty = Tx

Z_PRIOR = "uniform" #"uniform" / "informative" / "normal"
SIGMA_0_PRIOR = np.array([[1., 0.], [0., 1.]])

#pnas: diagonal [north: -45, south: 45]
#pnas: cardinal [north: 90, south: 0]
#ground-truth-shared: cardinal [north: 180, south: 0]
NORTH_ALPHA = 90
SOUTH_ALPHA = 0

NUM_OF_DATASETS = 10


In [ ]:
datasets = []
mllh_1x2D = []
mllh_2x2Ds = []
mllh_2x1D = []
mllh_2x1Doc = []

for i in range(NUM_OF_DATASETS):
    data1 = helper.generate_data(Tx, alpha=NORTH_ALPHA, context_value=0, z_prior_type=Z_PRIOR, sigma_reward=SIGMA_R)
    data2 = helper.generate_data(Ty, alpha=SOUTH_ALPHA, context_value=1, z_prior_type=Z_PRIOR, sigma_reward=SIGMA_R)

    true_data = helper.concatenate_data(data1, data2)
    #interleaved_indices = helper.riffle(np.arange(Tx+Ty)[:Tx],np.arange(Tx+Ty)[Tx:])
    #true_data = helper.reorder_data(true_data,interleaved_indices)

    mllh_1x2D.append(models.mllh_analytic_1x2D(true_data, SIGMA_R_MODEL, Sigma_0 = SIGMA_0_PRIOR))
    mllh_2x2Ds.append(models.mllh_analytic_2x2D_shared(true_data, SIGMA_R_MODEL, Sigma_0 = SIGMA_0_PRIOR))
    mllh_2x1D.append(models.mllh_analytic_2x1D(true_data, SIGMA_R_MODEL))
    mllh_2x1Doc.append(models.mllh_analytic_2x1D_observed_context(true_data, SIGMA_R_MODEL))
    datasets.append(true_data)

In [ ]:
plt.plot([mllh_1x2D[0],mllh_2x2Ds[0]])
plt.yscale('log')

In [ ]:
helper.plot_data(datasets[0], limit=1)

In [ ]:
plt.plot([mllh_2x1D,mllh_2x1Doc,mllh_1x2D,mllh_2x2Ds])
plt.yscale('log')
plt.xlabel('0 = 2x1D, 1 = 2x1D-observed context, 2 = 1x2D, 3 = 2x2D-shared')
plt.ylabel('mllh')
plt.show()

In [ ]:
plt.plot(np.array([mllh_2x1D,mllh_2x1Doc,mllh_1x2D,mllh_2x2Ds]).T)
plt.yscale('log')
plt.legend(['2x1D','2x1D observed context','1x2D','2x2D shared'])
plt.ylabel('mllh at T')
plt.xlabel('dataset index')
plt.show()

In [ ]:
posterior_2x2ds = models.mllh_analytic_2x2D_shared(true_data, SIGMA_R_MODEL, Sigma_0 = SIGMA_0_PRIOR, return_posterior=True)['mu_is']
posterior_1x2d = models.mllh_analytic_1x2D(true_data, SIGMA_R_MODEL, Sigma_0 = SIGMA_0_PRIOR, return_posterior=True)['mu_is']

In [ ]:
ax = plt.axes()
[ax.arrow(0, 0, sample[0], sample[1], head_width=0.02, head_length=0.05, fc='k', ec='k') for sample in posterior_2x2ds]
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
helper.plot_data(true_data, limit=1)

In [ ]:
ax = plt.axes()
ax.arrow(0, 0, posterior_2x2ds[-1][0], posterior_2x2ds[-1][1], head_width=0.02, head_length=0.05, fc='b', ec='b')
ax.arrow(0, 0, posterior_1x2d[-1][0], posterior_1x2d[-1][1], head_width=0.02, head_length=0.05, fc='g', ec='g') 
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.gca().set_aspect('equal')
plt.legend(('2x2D-S','1x2D'))
plt.show()

In [ ]:
plt.plot(np.array(posterior_2x2ds))
plt.gca().legend(('gamma_1','gamma_2'))
plt.show()

In [ ]:
helper.gamma_from_alpha(0)